In [87]:
#import common librarie
from zipfile import ZipFile 
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
from itertools import groupby
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



In [9]:
#import libraries to handle classic chinese word
from jiayan import load_lm
from jiayan import CharHMMTokenizer

# Baseline Model

## Load Data

In [22]:
file_name = "poems_with_tags.zip"
poem_df = pd.read_csv('poems_with_tags.zip', compression='zip', header=0, quotechar='"')
poem_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']"
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']"
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']"
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']"
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']"


In [65]:
poem_df['first_tag'] = poem_df \
.apply(lambda row : row.tags_list[1:-1]
       .replace("'","")
       .replace(",", "")
       .split(" ")[0], axis=1)

## Split Train and Test Data

In [68]:
poems_info = poem_df[['content','dynasty','author','title']]
tags = poem_df[['first_tag']]
X_train, X_test, y_train, y_test = train_test_split(poems_info, tags, test_size=0.33)

In [69]:
X_train.head()

,content,dynasty,author,title
4134,百战功成翻爱静，侯门渐欲似仙家。墙头雨细垂纤草，水面风回聚落花。井放辘轳闲浸酒，笼开鹦鹉报煎...,唐代,张蠙,夏日题老将林亭
3639,深居俯夹城，春去夏犹清。天意怜幽草，人间重晚晴。并添高阁迥，微注小窗明。越鸟巢干后，归飞体更轻。,唐代,李商隐,晚晴
3085,杳杳寒山道，落落冷涧滨。啾啾常有鸟，寂寂更无人。淅淅风吹面，纷纷雪积身。朝朝不见日，岁岁不知春。,唐代,寒山,杳杳寒山道
4458,繁如瑞雪压枝开，越岭吴溪免用栽。却是五侯家未识，春风不放过江来。,唐代,罗邺,梅花
2654,明月青山夜，高天白露秋。花庭开粉席，云岫敞针楼。石类支机影，池似泛槎流。暂惊河女鹊，终狎野人鸥。,唐代,李峤,同赋山居七夕


In [83]:
y_train['first_tag'].shape

(4099,)

## Build Model

In [71]:
lm = load_lm('jiayan.klm')

In [75]:
#Naive Bayes
tokenizer = CharHMMTokenizer(lm)
# print(list(tokenizer.tokenize(text)))


def tokenize_poem(some_list, f):
    return [f(x) for x in some_list]
X_train_tokenized = tokenize_poem(X_train['content'], lambda x: " ".join(list(tokenizer.tokenize(x))))
X_test_tokenized = tokenize_poem(X_test['content'], lambda x: " ".join(list(tokenizer.tokenize(x))))
count_vect = CountVectorizer()


In [ ]:
X_train_cv = count_vect.fit_transform(X_train_tokenized)
X_test_cv = count_vect.transform(X_test_tokenized)

In [ ]:
# word_freq_df = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names())
# top_words_df = pd.DataFrame(word_freq.sum()).sort_values(0, ascending=False)

In [76]:
naive_bayes = MultinomialNB()


## Train the Model

In [84]:
naive_bayes.fit(X_train_cv, y_train['first_tag'])
predictions = naive_bayes.predict(X_test_cv)

## Evaluation

In [90]:
print('Accuracy score: ', accuracy_score(y_test['first_tag'], predictions))
print('Precision score: ', precision_score(y_test['first_tag'], predictions, average='weighted'))
print('Recall score: ', recall_score(y_test['first_tag'], predictions, average='weighted'))

Accuracy score:  0.14554455445544554
Precision score:  0.10791065904170173
Recall score:  0.14554455445544554


/home/lindayang16/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
